## Import necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/OCR/')


In [3]:
!pip install easyocr
!pip install datasets
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
import os
import pandas as pd
import numpy as np
import easyocr
import cv2
import PIL
import urllib
import torch
from sklearn.preprocessing import LabelEncoder
import concurrent.futures
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import BertTokenizerFast, AutoModelForTokenClassification, BertTokenizer
from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration

from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from transformers import BertModel
from datasets import Dataset
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from src.utils import download_images
from src.utils import download_image
from src.constants import allowed_units
#utilizing GPU
device = "cuda" if torch.cuda.is_available() else "cpu"


## Getting familiar with dataset

In [5]:
df = pd.read_csv("/content/drive/MyDrive/OCR/dataset/train.csv")
df.shape

(263859, 4)

In [6]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [7]:
df.isna().sum()

,0
image_link,0
group_id,0
entity_name,0
entity_value,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [9]:
df["entity_value"].value_counts()

,count
entity_value,
100.0 gram,2438
100 gram,2117
30.0 centimetre,1809
20.0 centimetre,1705
10.0 centimetre,1663
...,...
28.07 inch,1
38.8 millimetre,1
12.66 inch,1


In [10]:
df["entity_name"].value_counts()

,count
entity_name,
item_weight,102786
depth,45127
width,44183
height,43597
voltage,9466
wattage,7755
item_volume,7682
maximum_weight_recommendation,3263


In [11]:
# num_samples = 3000

# sampled_df = df.groupby("entity_name", group_keys=False).apply(lambda x: x.sample(min(len(x), num_samples))).reset_index(drop=True)
# sampled_df["entity_name"].value_counts()

In [12]:
# sampled_df.head()

In [13]:
# sampled_df.shape

In [14]:
def show_4_images(df):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # Creating 2x2 grid
    axes = axes.flatten()  # Flatten the 2D array to a list
    for i in range(4):
        num = np.random.randint(0, len(df))
        image = PIL.Image.open(urllib.request.urlopen(df.image_link[num]))
        axes[i].imshow(image)
        axes[i].set_title(f"{df.entity_name[num]} = {df.entity_value[num]}")
    plt.show();

In [15]:
# show_4_images(sampled_df)

## OCR Experiment

In [16]:
def extract_text_from_image(reader, image_path):
    """
    Extract text from a single image with error handling.

    Args:
        reader (easyocr.Reader): EasyOCR reader instance
        image_path (str): Path to the image file

    Returns:
        tuple: (image_path, extracted_text)
    """
    try:
        read = reader.readtext(image_path, detail=0, paragraph=True)
        sentences = "--".join(read) if read else ""
        return image_path, sentences
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return image_path, ""

def read_text_from_images(df, max_workers=None):
    """
    Parallelize text extraction from images with error handling.

    Args:
        df (pd.DataFrame): DataFrame with 'image_link' column
        max_workers (int, optional): Number of parallel workers. Defaults to None (system default).

    Returns:
        pd.DataFrame: DataFrame with added 'text' column
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Initialize EasyOCR reader
    reader = easyocr.Reader(["en"], gpu=True)

    # Use ThreadPoolExecutor for I/O-bound tasks like image processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create a list of futures for each image
        futures = [
            executor.submit(extract_text_from_image, reader, image_path)
            for image_path in result_df["image_link"]
        ]

        # Create a dictionary to store results
        text_results = {}

        # Collect results as they complete
        for future in concurrent.futures.as_completed(futures):
            image_path, text = future.result()
            text_results[image_path] = text

        # Populate the 'text' column
        result_df["text"] = result_df["image_link"].map(text_results)

    return result_df

In [17]:
# sampled_df = read_text_from_images(sampled_df)
# sampled_df.head()

In [18]:
# sampled_df[["entity_value", "text"]]

In [19]:
# sampled_df.to_csv("/content/drive/MyDrive/OCR/sampled_df.csv", index=False)
df = pd.read_csv("/content/drive/MyDrive/OCR/sampled_df.csv")
df.head(10)

,image_link,group_id,entity_name,entity_value,text
0,https://m.media-amazon.com/images/I/518u672dVJ...,302672,depth,100.0 millimetre,Qualite premium Premium quality--100 mm--GARAN...
1,https://m.media-amazon.com/images/I/510HieWlmq...,131797,depth,144.3 millimetre,Spec--123--Li--144.3 mm--1 2930 41mm mm -| 20 ...
2,https://m.media-amazon.com/images/I/51n1LYjjy6...,631896,depth,6.3 inch,60 kg--10 kg--0 2--0--6
3,https://m.media-amazon.com/images/I/61+LRqVMVj...,354796,depth,3.6 metre,SPIRAL WOUND CARBON FIBER CLOTH--3.6M--3000g--...
4,https://m.media-amazon.com/images/I/51NZZAtqM5...,221399,depth,200.0 centimetre,450LB--0 1--78.8in/ 200cm--15.8in/ = 40cm
5,https://m.media-amazon.com/images/I/615w9gkeQ+...,941818,depth,6.0 foot,SIZE INFORMATION--100mm/3.9'--1--I 4--55mm/6'
6,https://m.media-amazon.com/images/I/51P5ebbqIV...,630869,depth,19.0 centimetre,Bead size &mm--3e--bracelet circle length: 19cm
7,https://m.media-amazon.com/images/I/61AySyt52C...,208023,depth,29.0 centimetre,Passe-partoul Inclus--atmosphera--30 4 atmosph...
8,https://m.media-amazon.com/images/I/61BAaPrq3M...,243137,depth,2.1 metre,2.Im / 7ft--6
9,https://m.media-amazon.com/images/I/51eT9Zph1C...,386460,depth,100.0 millimetre,Omm 40


In [20]:
df.shape

(24000, 5)

In [21]:
df["entity_name"].value_counts()

,count
entity_name,
depth,3000
height,3000
item_volume,3000
item_weight,3000
maximum_weight_recommendation,3000
voltage,3000
wattage,3000
width,3000


In [22]:
# Preprocess text to clean it and ensure it's a string
def cleaning(text):
    if isinstance(text, str):
        text = text.replace('[', '').replace(']', '').replace("'", '')
        text=text.encode('ascii',errors='ignore').decode()
        text=text.replace('\n','')
        text=text.replace('\t','')
        text=text.replace('/r','')
        text=text.replace('/','')
        text=text.replace('(','')
        text=text.replace(')','')
        text=text.replace('?','')
        text=text.replace('!','')
        text=text.replace('@','')
        text=text.replace('<','')
        text=text.replace('>','')

        return text
    return ''  # Return an empty string if the input is not a string (e.g., NaN or None)

# Apply text cleaning to the DataFrame
df['text'] = df['text'].apply(cleaning)

In [23]:
df[["entity_value", "text"]]

,entity_value,text
0,100.0 millimetre,Qualite premium Premium quality--100 mm--GARAN...
1,144.3 millimetre,Spec--123--Li--144.3 mm--1 2930 41mm mm -| 20 ...
2,6.3 inch,60 kg--10 kg--0 2--0--6
3,3.6 metre,SPIRAL WOUND CARBON FIBER CLOTH--3.6M--3000g--...
4,200.0 centimetre,450LB--0 1--78.8in 200cm--15.8in = 40cm
...,...,...
23995,25.0 centimetre,8.2cm3.2in--25cmo 9in--4.5cm1.8in
23996,10.0 centimetre,Height weight--1Ocm 3.9inch--196g--13cm 5.1inc...
23997,14.0 centimetre,Magnetic design Makes it more convenient to ta...
23998,15.0 millimetre,0--3


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[["image_link", "group_id", "entity_name", "text"]], df["entity_value"], test_size=0.25, random_state=42)
X_train.shape, y_test.shape

((18000, 4), (6000,))

## BERT

The way it works is that we pass a sequence to our encoder and it decode a sequence. So, sequence to sequence would be our primarly goal.

In [25]:
# Define valid entity-unit mappings
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}


In [28]:
# # Load BERT model and tokenizer (use GPU)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased').to(device)

# # Function to get BERT embeddings (GPU-accelerated)
# def get_bert_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Return embedding as numpy array on CPU

# # List to store BERT embeddings
# bert_embeddings = []

# for text in df['text']:
#     if text:
#         embedding = get_bert_embedding(text)
#         bert_embeddings.append(embedding)
#     else:
#         bert_embeddings.append(None)

# # Add BERT embeddings to DataFrame
# df['embedding'] = bert_embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [29]:
# df.to_csv("/content/drive/MyDrive/OCR/embeded_df.csv", index=False)
df = pd.read_csv("/content/drive/MyDrive/OCR/embeded_df.csv")

In [30]:
df.head()

,image_link,group_id,entity_name,entity_value,text,embedding
0,https://m.media-amazon.com/images/I/518u672dVJ...,302672,depth,100.0 millimetre,Qualite premium Premium quality--100 mm--GARAN...,[[-1.82583287e-01 1.35435194e-01 3.87052983e...
1,https://m.media-amazon.com/images/I/510HieWlmq...,131797,depth,144.3 millimetre,Spec--123--Li--144.3 mm--1 2930 41mm mm -| 20 ...,[[ 7.61073455e-02 2.05337554e-01 5.90421081e...
2,https://m.media-amazon.com/images/I/51n1LYjjy6...,631896,depth,6.3 inch,60 kg--10 kg--0 2--0--6,[[-2.44503871e-01 -7.86046609e-02 2.52212077e...
3,https://m.media-amazon.com/images/I/61+LRqVMVj...,354796,depth,3.6 metre,SPIRAL WOUND CARBON FIBER CLOTH--3.6M--3000g--...,[[ 3.17186825e-02 -1.41666800e-01 3.21863711e...
4,https://m.media-amazon.com/images/I/51NZZAtqM5...,221399,depth,200.0 centimetre,450LB--0 1--78.8in 200cm--15.8in = 40cm,[[-2.69734919e-01 7.67771006e-02 4.33013499e...


In [31]:
X_train['training_input'] = X_train[['entity_name','text']].agg(' | '.join, axis=1)
X_test['training_input'] = X_test[['entity_name','text']].agg(' | '.join, axis=1)

In [32]:
X_train

,image_link,group_id,entity_name,text,training_input
11152,https://m.media-amazon.com/images/I/61uvqEzWJ5...,483370,item_weight,Doll Size--55cm--weight : 135g,item_weight | Doll Size--55cm--weight : 135g
19068,https://m.media-amazon.com/images/I/8164dXsheR...,179080,wattage,~SINJN Jla--Toxihimang TZ--LARGE CAPACITY 36V ...,wattage | ~SINJN Jla--Toxihimang TZ--LARGE CAP...
14361,https://m.media-amazon.com/images/I/61Qc2x+HqV...,969033,maximum_weight_recommendation,SIZE MEASUREMENTS--SIZE--Lx W x H--WEIGHT--Sma...,maximum_weight_recommendation | SIZE MEASUREME...
16141,https://m.media-amazon.com/images/I/71bdRVHACT...,709627,voltage,DIMENSIONS--commilled building a thium NowtiPo...,voltage | DIMENSIONS--commilled building a thi...
7033,https://m.media-amazon.com/images/I/810wULYGjD...,120569,item_volume,POWERFUL--OEC FORMULA Highest quality active i...,item_volume | POWERFUL--OEC FORMULA Highest qu...
...,...,...,...,...,...
21575,https://m.media-amazon.com/images/I/61Eq4gCO6i...,347320,width,"pellon Yunt Unnl--65 Pellon"" Stabilizer 1 Craf...","width | pellon Yunt Unnl--65 Pellon"" Stabilize..."
5390,https://m.media-amazon.com/images/I/51gZd8tlYY...,180726,height,"Square Length 1-716""to1-58""--Fit Both Sqare & ...","height | Square Length 1-716""to1-58""--Fit Both..."
860,https://m.media-amazon.com/images/I/51iAsnmUSl...,965518,depth,PRODUCT SIZE 2cmo.78in { 5cm1_ 2--13.5cm5.3in,depth | PRODUCT SIZE 2cmo.78in { 5cm1_ 2--13.5...
15795,https://m.media-amazon.com/images/I/611f3skPyz...,752266,voltage,Jaquon LIGHTING Jaquar LED Down Light Model No...,voltage | Jaquon LIGHTING Jaquar LED Down Ligh...


In [34]:
# Text Cleaning Function
def cleaning(text):
    if not isinstance(text, str):
        return ''

    # Comprehensive text cleaning
    cleaning_patterns = [
        ('[', ''), (']', ''), ("'", ''),
        ('\n', ''), ('\t', ''), ('/r', ''),
        ('/', ''), ('(', ''), (')', ''),
        ('?', ''), ('!', ''), ('@', ''),
        ('<', ''), ('>', '')
    ]

    for pattern, replacement in cleaning_patterns:
        text = text.replace(pattern, replacement)

    # Remove non-ASCII characters
    text = text.encode('ascii', errors='ignore').decode()

    return text.strip()

# Data Preparation Function
def prepare_data(train_path, test_path):
    # Load datasets
    df_train = pd.read_csv(train_path)
    df_train = df_train[:18000]
    df_test = pd.read_csv(test_path)
    df_test = df_test[6000:]

    # Clean text
    df_train['text'] = df_train['text'].apply(cleaning)
    df_test['text'] = df_test['text'].apply(cleaning)

    # Create training input
    df_train['training_input'] = df_train[['entity_name', 'text']].agg(' | '.join, axis=1)
    df_test['training_input'] = df_test[['entity_name', 'text']].agg(' | '.join, axis=1)

    return df_train, df_test

# Training Function
def train_bart_model(X_train, y_train, epochs=20, batch_size=16, learning_rate=1e-4, weight_decay=0.01):
    # Initialize tokenizer and model
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

    # Determine device
    model.to(device)

    # Tokenize inputs
    def tokenize_function(text_list):
        return tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=512)

    input_tokens = [tokenize_function(x)['input_ids'].squeeze(0) for x in X_train]
    target_tokens = [tokenize_function(y)['input_ids'].squeeze(0) for y in y_train]

    # Move tokens to device
    input_tokens = [x.to(device) for x in input_tokens]
    target_tokens = [y.to(device) for y in target_tokens]

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3
    )

    # Training loop
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        print(f'Epoch {epoch + 1}/{epochs}')

        for i in tqdm(range(0, len(input_tokens), batch_size), desc="Training Progress", unit="batch"):
            inputs_batch = torch.nn.utils.rnn.pad_sequence(
                input_tokens[i:i + batch_size],
                batch_first=True,
                padding_value=tokenizer.pad_token_id
            )
            targets_batch = torch.nn.utils.rnn.pad_sequence(
                target_tokens[i:i + batch_size],
                batch_first=True,
                padding_value=tokenizer.pad_token_id
            )

            optimizer.zero_grad()

            outputs = model(input_ids=inputs_batch, labels=targets_batch)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1} completed. Average Loss: {total_loss / len(input_tokens)}')

    return model, tokenizer

# Add these imports at the top of your script
from sklearn.metrics import precision_recall_fscore_support

# Modify the evaluate_model function
def evaluate_model(model, tokenizer, X_test, y_test):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    smooth_fn = SmoothingFunction().method4

    input_tokens_test = [
        tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)['input_ids'].squeeze(0).to(device)
        for text in X_test
    ]

    predicted_values = []
    actual_values = list(y_test)

    total_bleu_score = 0
    correct_predictions = 0

    batch_size = 8

    for i in tqdm(range(0, len(input_tokens_test), batch_size), desc="Evaluating"):
        inputs_batch = torch.nn.utils.rnn.pad_sequence(
            input_tokens_test[i:i + batch_size],
            batch_first=True,
            padding_value=tokenizer.pad_token_id
        )

        with torch.no_grad():
            generated_ids = model.generate(inputs_batch, max_length=20)

        for idx, generated_id in enumerate(generated_ids):
            generated_text = tokenizer.decode(generated_id, skip_special_tokens=True)
            predicted_values.append(generated_text)

            actual_text = actual_values[i + idx]
            reference = [actual_text.split()]
            candidate = generated_text.split()

            bleu_score = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
            total_bleu_score += bleu_score

            if generated_text == actual_text:
                correct_predictions += 1

    # Compute metrics
    average_bleu_score = total_bleu_score / len(predicted_values)
    accuracy = correct_predictions / len(predicted_values)


    # Print results
    print("\nSample Predictions:")
    for i in range(min(10, len(predicted_values))):
        print(f"Predicted: {predicted_values[i]}, Actual: {actual_values[i]}")
        reference = [actual_values[i].split()]
        candidate = predicted_values[i].split()
        sample_bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        print(f"BLEU score for sample {i}: {sample_bleu:.4f}")

    print(f"\nExact Match Accuracy: {accuracy * 100:.2f}%")
    print(f"Average BLEU Score: {average_bleu_score:.4f}")

    return predicted_values, {
        'accuracy': accuracy,
        'bleu_score': average_bleu_score,

    }

# Main Execution Pipeline
def main():
    # Paths - update these to your specific file paths
    train_path = '/content/drive/MyDrive/OCR/embeded_df.csv'
    test_path = '/content/drive/MyDrive/OCR/embeded_df.csv'

    # Prepare data
    df_train, df_test = prepare_data(train_path, test_path)

    # Split training data
    X = df_train['training_input']
    y = df_train['entity_value']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20, shuffle=True)

    # Train model
    model, tokenizer = train_bart_model(X_train, y_train)

    # Evaluate model
    predictions = evaluate_model(model, tokenizer, X_test, y_test)



if __name__ == "__main__":
    main()

Epoch 1/20


Training Progress: 100%|██████████| 900/900 [01:54<00:00,  7.86batch/s]


Epoch 1 completed. Average Loss: 0.03712280699776278
Epoch 2/20


Training Progress: 100%|██████████| 900/900 [01:54<00:00,  7.86batch/s]


Epoch 2 completed. Average Loss: 0.023260172915955384
Epoch 3/20


...

Epoch 19 completed. Average Loss: 0.002764047009833222
Epoch 20/20


Training Progress: 100%|██████████| 900/900 [01:54<00:00,  7.86batch/s]


Epoch 20 completed. Average Loss: 0.0022182592589767105


Evaluating: 100%|██████████| 450/450 [01:18<00:00,  5.73it/s]


Sample Predictions:
Predicted: 70.0 centimetre, Actual: 70.0 centimetre
BLEU score for sample 0: 0.2214
Predicted: 0.95 fluid ounce, Actual: 0.34 fluid ounce
BLEU score for sample 1: 0.2118
Predicted:  410.0 gram, Actual: 410.0 gram
BLEU score for sample 2: 0.2214
Predicted: 40.0 centimetre, Actual: 30.0 centimetre
BLEU score for sample 3: 0.0675
Predicted: 45.0 centimetre, Actual: 45.0 centimetre
BLEU score for sample 4: 0.2214
Predicted: [400.0, 240.0] gram, Actual: 240.0 gram
BLEU score for sample 5: 0.0725
Predicted: 25.0 centimetre, Actual: 25.0 centimetre
BLEU score for sample 6: 0.2214
Predicted: 60.0 millilitre, Actual: 60.0 fluid ounce
BLEU score for sample 7: 0.0410
Predicted: 25.0 gram, Actual: [2007.0, 2009.0] gram
BLEU score for sample 8: 0.0410
Predicted: 5.0 gram, Actual: 0.3333333333333333 ton
BLEU score for sample 9: 0.0000

Exact Match Accuracy: 46.22%
Average BLEU Score: 0.1484

Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
